In [ ]:
# Import Pandas, for working with tabular data 
import pandas as pd


In [ ]:
# Import a data file as a pandas DataFrame object
data = pd.read_csv('data_S03_INTEGRATE_fMRI1_Retrieval.txt',sep='\t')


In [ ]:
# The .head() method shows the header for the table (first few rows)
data.head()


In [ ]:
data.RT[0:10]

In [ ]:
# Import some other packages
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt


In [ ]:
# Let's make a histogram to do a quick visual inspection of the data
plt.hist(data.RT)
plt.show()

In [ ]:
# Zscoring
mu = np.mean(data.RT)
sd = np.std(data.RT)
#print(mu)
#print(sd)

# Calculate the zscore of each value in the RT distribution
(data.RT[0:10]-mu)/sd

In [ ]:
# Can also use other built-in functions
zscores = st.zscore(data.RT)
print(zscores)

In [ ]:
# Identifying correct trials.
data

In [ ]:
# Priming analysis
# Boundary vs NonBoundary: data.BoundaryType == 'B'
# Consecutive items only: data.PairType=='P23'
# data.PrimeType=='P2' - data.PrimeType=='P1'
# data.Accuracy==1
# zscore < nSDs (5)

priming_b = []
priming_nb = []
priming_baseline = []
nSDs = 5

for ctr,iRT in enumerate(data.RT):
    
    # if it's the first trial, pass
    if ctr==0:
        pass
    else:
        # Check that the current trial satisfies our criteria
        #  First check is PrimeType=='P2'
        validcurrent = data.PairType[ctr]=='P23' and data.PrimeType[ctr]=='P2' and \
            data.Accuracy[ctr]==1 and zscores[ctr] < nSDs
        validprev = data.PairType[ctr-1]=='P23' and data.PrimeType[ctr-1]=='P1' and \
            data.Accuracy[ctr-1]==1 and zscores[ctr-1] < nSDs
        
        if validcurrent & validprev:
            if data.BoundaryType[ctr]=='B' and data.BoundaryType[ctr-1]=='B':
                priming_b = np.append(priming_b,data.RT[ctr]-data.RT[ctr-1])
            elif data.BoundaryType[ctr]=='NB' and data.BoundaryType[ctr-1]=='NB':
                priming_nb = np.append(priming_nb,data.RT[ctr]-data.RT[ctr-1])
        else:
            validbaseline = data.ItemOldNew[ctr]=='old' and \
                data.ItemOldNew[ctr-1]=='old' and \
                data.Accuracy[ctr]==1 and data.Accuracy[ctr-1]==1 and \
                zscores[ctr] < nSDs and zscores[ctr-1] < nSDs
            if validbaseline:
                priming_baseline = np.append(priming_baseline,data.RT[ctr]-data.RT[ctr-1])
            

In [ ]:
print(priming_b.shape)
print(priming_b)
print(priming_nb.shape)
print(priming_nb)
print(priming_baseline.shape)
print(priming_baseline)

In [ ]:
print(np.nanmean(priming_b))
print(np.nanmean(priming_nb))
print(np.nanmean(priming_baseline))